In [5]:
from AttentionModule import Conv2d_Attn

import torch
from torch import nn
from torchvision import models, datasets, transforms
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

import re
import numpy as np

In [6]:
resnet_pretrained = models.resnet50(pretrained=True)

In [7]:
nn.Conv2d = Conv2d_Attn
resnet_attn = models.resnet50()

In [8]:
resnet_attn.load_state_dict(resnet_pretrained.state_dict(), strict=False)

In [9]:
# Change batchnorm behavior
# resnet_attn = resnet_attn.eval() 
# Don't want to do that because bn needs to be re-trained as well

In [10]:
# This block turns 'layer1.0.downsample.0.weight' to 'layer1[0].downsample[0].weight'
param_keys = list(resnet_attn.state_dict().keys())
formatted_keys = []
for k in param_keys:
    found = re.findall(r'\.[\d]{1,2}\.', k)
    if len(found):
        for f in found:
            k = k.replace(f, '[{}].'.format(f.strip('.')))
    formatted_keys.append(k)

In [13]:
# This block turn off gradient up for all params except attn_weights
def turn_off_grad_except(lst=[]):
    for k in formatted_keys:
        obj = eval('resnet_attn.'+k)
        for kw in lst:
            if not kw in k:
                obj.requires_grad = False
            else:
                obj.requires_grad = True

In [14]:
turn_off_grad_except(['attn_weights', 'bn'])
resnet_attn.fc = nn.Linear(resnet_attn.fc.in_features, 144)

Start training

In [15]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose(
    [transforms.ToTensor(),
     normalize])

trainset = torchvision.datasets.ImageFolder(root='./data/train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

In [16]:
cls_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet_attn.parameters()))

In [17]:
# resnet_attn = resnet_attn.cuda()

In [18]:
total_attn_params = 0
for k in formatted_keys:
    obj = eval('resnet_attn.'+k)
    if 'attn_weights' in k:
        total_attn_params += np.prod(obj.shape)
print("Total number of attention parameters", total_attn_params)

Total number of attention parameters 26560


We want the attention parameters to diverge from 1, therefore we penalize element-wise square loss as $\lambda (1 \times \text{# params} - (x - 1)^2)$

In [31]:
_lambda = 1e-2

In [32]:
def get_params_objs(name, net='resnet_attn'):
    res = []
    for k in formatted_keys:
        obj = eval(f'{net}.'+k)
        if name in k:
            res.append(obj)
    return res

In [43]:
def compute_attn_loss(n_params=26560):
    attns = get_params_objs('attn_weights')
    penality = sum([torch.pow(t - 1,2).mean() for t in attns])
    return _lambda*(n_params - penality)

In [44]:
compute_attn_loss()

Variable containing:
 265.6000
[torch.FloatTensor of size 1]

In [46]:
print_every = 200

In [47]:
running_loss = 0.0
running_attn_loss = 0.0
for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    inputs, labels = Variable(inputs), Variable(labels)

    optimizer.zero_grad()
    outputs = resnet_attn(inputs)
    loss = cls_criterion(outputs, labels)
    attn_loss = compute_attn_loss()
    loss += attn_loss
    
    loss.backward()
    optimizer.step()
    

    running_loss += loss.data[0]
    running_attn_loss += attn_loss.data[0]
    
    if i % print_every == 0:
        print('[%5d] avg loss: %.3f, attn_loss: %.5f ' %
              (i + 1, running_loss/print_every, running_attn_loss/print_every))
        running_loss = 0.0
        running_attn_loss = 0.0

[    1] avg loss: 271.873, attn_loss: 265.60001 
[    2] avg loss: 271.564, attn_loss: 265.60001 


Process Process-10:
Process Process-9:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/simonmo/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/simonmo/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/simonmo/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/simonmo/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/simonmo/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/Users/simonmo/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/Users/simonmo/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
   

KeyboardInterrupt: 